In [1]:
import torch
from torch import nn
from torchinfo import summary

![](./images/mobilenet_v2/mobilenet_v2_diagram.png)
![](./images/mobilenet_v2/mobilenet_v2_block.png)

In [13]:
class DepSepConv(nn.Module):
    def __init__(self, in_chaneels, out_channels, stride):
        super().__init__()

        self.depthwise = nn.Sequential(
            nn.Conv2d(in_chaneels, in_chaneels, 3, stride=stride, padding=1, groups=in_chaneels, bias=False),
            nn.BatchNorm2d(in_chaneels),
            nn.ReLU6()
        )

        self.seperate = nn.Sequential(
            nn.Conv2d(in_chaneels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels)
            # No Activation at MobileNet_V2!!
        )
    
    def forward(self, x):
        x = self.depthwise(x)
        x = self.seperate(x)
        return x

In [14]:
class InvertedBlock(nn.Module):
    def __init__(self, in_channels, expansion_channels, out_channels, stride):
        super().__init__()

        if stride==1 and in_channels==out_channels:
            self.use_skip = True
        else:
            self.use_skip = False

        layers = []
        if in_channels != expansion_channels: # expansion이 존재할 때만 처음 1x1 conv를 채널 늘리기 위해 사용
            layers += [
                nn.Sequential(
                    nn.Conv2d(in_channels, expansion_channels, 1, bias=False),
                    nn.BatchNorm2d(expansion_channels),
                    nn.ReLU6()
                )
            ]

        layers += [DepSepConv(expansion_channels, out_channels, stride=stride)]

        self.residual = nn.Sequential(*layers)
    
    def forward(self, x):
        if self.use_skip==True:
            output = x + self.residual(x)
        else:
            output = self.residual(x)
        
        return output

![](./images/mobilenet_v2/mobilenet_v2_table.png)

In [15]:
class MobileNetV2(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU6()
        )

        self.in_channels = 32        
        bottleneck_layers = []
        t_c_n_s = [
            [1, 16,  1, 1],
            [6, 24,  2, 2],
            [6, 32,  3, 2],
            [6, 64,  4, 2],
            [6, 96,  3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1]
        ]

        for t, c, n, s in t_c_n_s:
            bottleneck_layers += self.make_bottleneck(t, c, n, s)       

        self.bottleneck_layers = nn.Sequential(*bottleneck_layers)   

        self.last_conv = nn.Sequential(
            nn.Conv2d(self.in_channels, 1280, 1, bias=False),
            nn.BatchNorm2d(1280),
            nn.ReLU6()
        )

        self.GlobalAvgPool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bottleneck_layers(x)
        x = self.last_conv(x)

        x = self.GlobalAvgPool(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x

    def make_bottleneck(self, t, c, n, s):
        layers = []

        for idx in range(n):                
            expansion_channels = self.in_channels * t
            stride = s if idx==0 else 1

            layers += [InvertedBlock(self.in_channels, expansion_channels, out_channels=c, stride=stride)]
            self.in_channels = c   

        return layers         

model = MobileNetV2(num_classes=1000)
summary(model, (2, 3, 224, 224))

Layer (type:depth-idx)                             Output Shape              Param #
MobileNetV2                                        [2, 1000]                 --
├─Sequential: 1-1                                  [2, 32, 112, 112]         --
│    └─Conv2d: 2-1                                 [2, 32, 112, 112]         864
│    └─BatchNorm2d: 2-2                            [2, 32, 112, 112]         64
│    └─ReLU6: 2-3                                  [2, 32, 112, 112]         --
├─Sequential: 1-2                                  [2, 320, 7, 7]            --
│    └─InvertedBlock: 2-4                          [2, 16, 112, 112]         --
│    │    └─Sequential: 3-1                        [2, 16, 112, 112]         896
│    └─InvertedBlock: 2-5                          [2, 24, 56, 56]           --
│    │    └─Sequential: 3-2                        [2, 24, 56, 56]           5,136
│    └─InvertedBlock: 2-6                          [2, 24, 56, 56]           --
│    │    └─Sequential: 3-3   